In [3]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [4]:
from sklearn.preprocessing import LabelBinarizer
from loader_util.nn.conv import MiniGoogLeNet
from loader_util.callbacks import TrainingMonitor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import numpy as np
import os

In [5]:
# create the learning rate decay function
TOTAL_EPOCHS = 70
INIT_LR = 0.005


def poly_decay(epoch):
    max_epochs = TOTAL_EPOCHS
    base_lr = INIT_LR
    power = 1.0

    alpha = base_lr * (1 - (epoch / float(max_epochs))) ** power
    return alpha

In [6]:
# script constants
model_output_path = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap11/artifacts/model_28nov"
artifacts_output_dir = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap11/artifacts"

In [7]:
print(f"[INFO] loading cifar10 data......")
((trainx, trainy), (testx, testy)) = cifar10.load_data()
trainx = trainx.astype("float")
testx = testx.astype("float")

# apply mean subtraction
mean = np.mean(trainx, axis=0)
trainx -= mean
testx -= mean

[INFO] loading cifar10 data......
170498071/170498071 [==============================] - 6s 0us/step


In [8]:
# convert the labels to encoded versions
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.transform(testy)

In [9]:
# init the data augmentation
aug = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [10]:
# construct the set of callbacks
fig_path = os.path.sep.join([artifacts_output_dir, f"{os.getpid()}.png"])
json_path = os.path.sep.join([artifacts_output_dir, f"{os.getpid()}.json"])
callbacks = [TrainingMonitor(fig_path, json_path),
             LearningRateScheduler(poly_decay)]

In [11]:
# init the opt and the model
print(f"[INFO] compiling model......")
opt = SGD(learning_rate=INIT_LR,
          momentum=0.9)
model = MiniGoogLeNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

[INFO] compiling model......


In [12]:
# train the network
print(f"[INFO] training the network......")
model.fit_generator(aug.flow(trainx, trainy, batch_size=64),
                    validation_data=(testx, testy),
                    steps_per_epoch=len(trainx) // 64,
                    epochs=TOTAL_EPOCHS,
                    callbacks=callbacks,
                    verbose=1)
# saving the model
print(f"[INFO] serializing the model......")
model.save(model_output_path)

[INFO] training the network......


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/70
781/781 [==============================] - 40s 36ms/step - loss: 1.5753 - accuracy: 0.4473 - val_loss: 1.2303 - val_accuracy: 0.5808 - lr: 0.0050
Epoch 2/70
781/781 [==============================] - 28s 36ms/step - loss: 1.1471 - accuracy: 0.5938 - val_loss: 1.0688 - val_accuracy: 0.6381 - lr: 0.0049
Epoch 3/70
781/781 [==============================] - 28s 35ms/step - loss: 1.0033 - accuracy: 0.6467 - val_loss: 0.8948 - val_accuracy: 0.6831 - lr: 0.0049
Epoch 4/70
781/781 [==============================] - 28s 36ms/step - loss: 0.9011 - accuracy: 0.6834 - val_loss: 0.8071 - val_accuracy: 0.7241 - lr: 0.0048
Epoch 5/70
781/781 [==============================] - 28s 36ms/step - loss: 0.8197 - accuracy: 0.7130 - val_loss: 0.8205 - val_accuracy: 0.7177 - lr: 0.0047
Epoch 6/70
781/781 [==============================] - 28s 35ms/step - loss: 0.7639 - accuracy: 0.7335 - val_loss: 0.7822 - val_accuracy: 0.7307 - lr: 0.0046
Epoch 7/70
781/781 [==============================] - 28s 